<a href="https://colab.research.google.com/github/BuxGuerra/TPICD/blob/main/analiseFinalista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prevendo finalistas da NBA

O nosso objetivo neste notebook é prever se um time da NBA irá chegar à final dos playofs analisando suas estatísticas durante a temporada regular.

Começaremos com a preparação e o pré-processamento dos dados.

In [170]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import (confusion_matrix, accuracy_score)
from sklearn.linear_model import LogisticRegression
import seaborn
import matplotlib.pyplot as plt
import statsmodels.api as sm

##Preparação dos dados

###Concatenar bases e rotular

Nessa etapa temos dois objetivos:



*   Identificar os finalistas
*   Unificar as bases de dados

Como dito no notebook importanciaCaracteristicas.ipynb, cada base de dados que temos representa uma temporada regular da NBA. As linhas das tabelas representam os times e as colunas nos fornecem estatísticas sumarizadas dos times ao longo da temporada regular.

In [171]:
!git clone https://github.com/BuxGuerra/TPICD.git

dfTemporada2010 = pd.read_csv("/content/TPICD/data/basesSeparadas/2010-11.csv")
dfTemporada2010

fatal: destination path 'TPICD' already exists and is not an empty directory.


,Unnamed: 0,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,0,1610612737,Atlanta Hawks,82,44,38,0.537,48.1,36.2,78.4,...,28,12,9,29,28,5,1,29,26,16
1,1,1610612738,Boston Celtics,82,56,26,0.683,48.2,36.9,75.8,...,29,4,22,3,27,10,15,22,23,6
2,2,1610612766,Charlotte Bobcats,82,34,48,0.415,48.3,35.0,77.6,...,24,18,21,28,8,29,8,13,29,24
3,3,1610612741,Chicago Bulls,82,62,20,0.756,48.4,37.1,80.3,...,2,9,17,17,5,26,9,24,20,2
4,4,1610612739,Cleveland Cavaliers,82,19,63,0.232,48.2,35.2,81.1,...,22,20,18,26,28,28,12,10,25,30
5,5,1610612742,Dallas Mavericks,82,57,25,0.695,48.1,37.4,78.8,...,14,2,12,23,24,2,4,23,11,8
6,6,1610612743,Denver Nuggets,82,50,32,0.610,48.1,38.4,80.6,...,11,10,16,13,24,27,18,1,1,7
7,7,1610612765,Detroit Pistons,82,30,52,0.366,48.5,37.3,81.1,...,30,17,2,16,30,15,7,27,22,23
8,8,1610612744,Golden State Warriors,82,36,46,0.439,48.4,39.6,85.9,...,19,7,23,2,12,6,24,30,7,21
9,9,1610612745,Houston Rockets,82,43,39,0.524,48.5,38.7,85.1,...,7,1,8,21,19,22,11,8,3,11


In [172]:
#Dicionário para identificar finalistas:
seasonFirstSecond = {'2010-11':('Dallas Mavericks','Miami Heat'),
                     '2011-12':('Miami Heat','Oklahoma City Thunder'),
                     '2012-13':('Miami Heat','San Antonio Spurs'),
                     '2013-14':('San Antonio Spurs','Miami Heat'),
                     '2014-15':('Golden State Warriors','Cleveland Cavaliers'),
                     '2015-16':('Cleveland Cavaliers','Golden State Warriors'),
                     '2016-17':('Golden State Warriors','Miami Heat'),
                     '2017-18':('Dallas Mavericks','Cleveland Cavaliers'),
                     '2018-19':('Toronto Raptors','Golden State Warriors'),
                     '2019-20':('Los Angeles Lakers','Miami Heat'),
                     '2020-21':('Milwaukee Bucks','Phoenix Suns'),
                     '2021-22':('Golden State Warriors','Boston Celtics'),
                     '2022-23':('Denver Nuggets','Miami Heat'),}

Para identificarmos os finalistas iremos adicionar uma nova coluna aos dados chamada 'FINAL'. Essa coluna terá valor 0 caso o time não tenha chego a final e valor 1 caso tenha chego a final.

In [173]:
def label(dfSeason, finalistas):
  dfSeason['FINAL'] = 0
  dfSeason['FINAL'] = np.where((dfSeason['TEAM_NAME'] == finalistas[0]) | (dfSeason['TEAM_NAME'] == finalistas[1]), 1, dfSeason['FINAL'])


Aproveitaremos o loop que aplica a função acima em todas as 13 bases de dados para também concatená-las em um dataframe apenas.

In [174]:
#Aplicar a função label e concatenar:

directory = '/content/TPICD/data/basesSeparadas'
dfSeasonTest = pd.read_csv("/content/TPICD/data/basesSeparadas/2010-11.csv")
dfFinal = pd.DataFrame(columns=dfSeasonTest.columns)


for fileName in os.listdir(directory):
    if(fileName != '.ipynb_checkpoints'):
      filePath = os.path.join(directory, fileName)
      dfSeason = pd.read_csv(filePath)
      season = fileName.split('.csv')[0]
      label(dfSeason, seasonFirstSecond[season])
      dfFinal = pd.concat([dfFinal, dfSeason])

dfFinal.to_csv('TemporadasUnificadas.csv')

dfFinal

,Unnamed: 0,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,...,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,FINAL
0,0,1610612737,Atlanta Hawks,82,48,34,0.585,48.4,38.6,84.4,...,2,22,3,5,14,7,29,12,7,0.0
1,1,1610612738,Boston Celtics,82,48,34,0.585,48.2,39.2,89.2,...,6,12,2,22,24,29,10,5,8,0.0
2,2,1610612751,Brooklyn Nets,82,21,61,0.256,48.2,38.2,84.4,...,15,19,19,25,18,2,28,26,28,0.0
3,3,1610612766,Charlotte Hornets,82,48,34,0.585,48.4,37.0,84.4,...,19,1,22,12,21,3,14,11,9,0.0
4,4,1610612741,Chicago Bulls,82,42,40,0.512,48.5,38.6,87.4,...,12,13,29,8,27,5,26,21,18,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,25,1610612758,Sacramento Kings,82,27,55,0.329,48.2,38.8,86.1,...,25,9,11,25,14,19,30,30,28,0.0
26,26,1610612759,San Antonio Spurs,82,47,35,0.573,48.1,39.0,85.4,...,15,4,14,4,2,1,18,27,8,0.0
27,27,1610612761,Toronto Raptors,82,59,23,0.720,48.4,41.3,87.4,...,6,6,18,2,18,27,16,4,2,0.0
28,28,1610612762,Utah Jazz,82,48,34,0.585,48.2,38.3,82.9,...,21,20,4,9,11,16,8,19,5,0.0


Verificar valores nulos:

In [175]:
dfFinal.isnull().sum()

Unnamed: 0         0
TEAM_ID            0
TEAM_NAME          0
GP                 0
W                  0
L                  0
W_PCT              0
MIN                0
FGM                0
FGA                0
FG_PCT             0
FG3M               0
FG3A               0
FG3_PCT            0
FTM                0
FTA                0
FT_PCT             0
OREB               0
DREB               0
REB                0
AST                0
TOV                0
STL                0
BLK                0
BLKA               0
PF                 0
PFD                0
PTS                0
PLUS_MINUS         0
GP_RANK            0
W_RANK             0
L_RANK             0
W_PCT_RANK         0
MIN_RANK           0
FGM_RANK           0
FGA_RANK           0
FG_PCT_RANK        0
FG3M_RANK          0
FG3A_RANK          0
FG3_PCT_RANK       0
FTM_RANK           0
FTA_RANK           0
FT_PCT_RANK        0
OREB_RANK          0
DREB_RANK          0
REB_RANK           0
AST_RANK           0
TOV_RANK     

###Pré-processamento

Iremos excluir algumas colunas que não irão nos ajudar, são elas: 'Unnamed: 0', 'TEAM_ID', 'TEAM_NAME' e 'GP'     (GP = Games played, igual para todos)

In [176]:
dfFinal = dfFinal.iloc[:,4:]
dfFinal

,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,...,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,FINAL
0,48,34,0.585,48.4,38.6,84.4,0.458,9.9,28.4,0.350,...,2,22,3,5,14,7,29,12,7,0.0
1,48,34,0.585,48.2,39.2,89.2,0.439,8.7,26.1,0.335,...,6,12,2,22,24,29,10,5,8,0.0
2,21,61,0.256,48.2,38.2,84.4,0.453,6.5,18.4,0.352,...,15,19,19,25,18,2,28,26,28,0.0
3,48,34,0.585,48.4,37.0,84.4,0.439,10.6,29.4,0.362,...,19,1,22,12,21,3,14,11,9,0.0
4,42,40,0.512,48.5,38.6,87.4,0.441,7.9,21.4,0.371,...,12,13,29,8,27,5,26,21,18,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,27,55,0.329,48.2,38.8,86.1,0.450,9.0,24.0,0.375,...,25,9,11,25,14,19,30,30,28,0.0
26,47,35,0.573,48.1,39.0,85.4,0.457,8.5,24.1,0.352,...,15,4,14,4,2,1,18,27,8,0.0
27,59,23,0.720,48.4,41.3,87.4,0.472,11.8,33.0,0.358,...,6,6,18,2,18,27,16,4,2,0.0
28,48,34,0.585,48.2,38.3,82.9,0.462,10.8,29.6,0.366,...,21,20,4,9,11,16,8,19,5,0.0


Também iremos separar o dataframe em dois, um que contém as estatísticas e o label de finalista e outro que contém os rankings das características e o label de finalista:

In [177]:
dfEstat = dfFinal.iloc[:,:25]
dfEstat['FINAL'] = dfFinal['FINAL']
dfRank = dfFinal.iloc[:,26:]

Além disso, entre W (wins), L (losses) e W_PCT (win percentage) basta mantermos uma delas, no caso W_PCT será mantida:

In [178]:
dfEstat = dfEstat.iloc[:,2:]
dfRank = dfRank.iloc[:,2:]



In [179]:
dfEstat.head()

,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,FINAL
0,0.585,48.4,38.6,84.4,0.458,9.9,28.4,0.350,15.6,20.0,...,25.6,15.0,9.1,5.9,5.0,19.1,18.3,102.8,3.6,0.0
1,0.585,48.2,39.2,89.2,0.439,8.7,26.1,0.335,18.5,23.5,...,24.2,13.7,9.2,4.2,5.5,21.9,21.0,105.7,3.2,0.0
2,0.256,48.2,38.2,84.4,0.453,6.5,18.4,0.352,15.7,20.7,...,22.3,14.8,7.6,4.0,5.2,18.0,18.4,98.6,-7.4,0.0
3,0.585,48.4,37.0,84.4,0.439,10.6,29.4,0.362,18.7,23.7,...,21.7,12.5,7.3,5.3,5.5,18.1,20.4,103.4,2.7,0.0
4,0.512,48.5,38.6,87.4,0.441,7.9,21.4,0.371,16.5,21.0,...,22.8,13.9,6.0,5.7,5.7,18.8,18.7,101.6,-1.5,0.0


In [180]:
dfRank.head()

,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,...,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,FINAL
0,7,11,9,15,11,6,7,15,29,30,...,2,22,3,5,14,7,29,12,7,0.0
1,7,23,7,1,24,12,11,28,8,12,...,6,12,2,22,24,29,10,5,8,0.0
2,28,26,17,17,13,26,27,13,27,27,...,15,19,19,25,18,2,28,26,28,0.0
3,7,9,24,16,27,4,4,8,7,11,...,19,1,22,12,21,3,14,11,9,0.0
4,14,3,11,2,22,21,24,3,23,26,...,12,13,29,8,27,5,26,21,18,0.0


##Análise

Inicialmente iremos buscar correlações entre os dados:

In [181]:
cormat = dfEstat.corr()
round(cormat,2)

,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,FINAL
W_PCT,1.00,0.04,0.30,-0.08,0.61,0.21,0.12,0.55,0.18,0.10,...,0.28,-0.27,0.18,0.25,-0.48,-0.21,0.10,0.32,0.96,0.32
MIN,0.04,1.00,-0.09,-0.11,-0.03,-0.18,-0.18,-0.06,0.09,0.09,...,-0.07,0.02,-0.02,0.02,0.00,0.10,0.10,-0.11,0.02,-0.03
FGM,0.30,-0.09,1.00,0.81,0.70,0.69,0.67,0.35,0.04,-0.08,...,0.77,-0.17,0.11,0.08,-0.20,-0.04,-0.05,0.93,0.32,0.13
FGA,-0.08,-0.11,0.81,1.00,0.15,0.67,0.71,0.02,-0.07,-0.13,...,0.56,-0.15,0.07,-0.01,0.08,0.06,-0.08,0.78,-0.05,-0.08
FG_PCT,0.61,-0.03,0.70,0.15,1.00,0.35,0.27,0.57,0.14,0.03,...,0.62,-0.10,0.11,0.15,-0.45,-0.13,0.01,0.63,0.61,0.31
FG3M,0.21,-0.18,0.69,0.67,0.35,1.00,0.99,0.38,0.02,-0.08,...,0.59,-0.12,-0.03,-0.08,-0.26,-0.04,-0.03,0.84,0.22,0.11
FG3A,0.12,-0.18,0.67,0.71,0.27,0.99,1.00,0.23,0.03,-0.05,...,0.56,-0.09,-0.02,-0.10,-0.19,-0.02,0.00,0.82,0.13,0.07
FG3_PCT,0.55,-0.06,0.35,0.02,0.57,0.38,0.23,1.00,-0.05,-0.18,...,0.35,-0.21,-0.04,0.05,-0.44,-0.12,-0.16,0.36,0.56,0.25
FTM,0.18,0.09,0.04,-0.07,0.14,0.02,0.03,-0.05,1.00,0.91,...,-0.11,0.08,-0.00,0.09,0.13,0.24,0.75,0.27,0.19,-0.03
FTA,0.10,0.09,-0.08,-0.13,0.03,-0.08,-0.05,-0.18,0.91,1.00,...,-0.20,0.22,0.05,0.09,0.19,0.27,0.83,0.14,0.11,-0.06


Percebe-se algumas correlações mais altas:

*   FG3M e FG3A = 0.99
*   W_PCT e PLUS_MINUS = 0.96
*   POINTS e FGM = 0.93
*   FTM e FTA = 0.91

In [182]:
#Excluir correlação entre as estatísticas:

dfEstatComp = dfEstat
dfRankComp = dfRank


dfEstat = dfEstat.drop(['FG3A', 'PLUS_MINUS', 'FGM', 'FTA'], axis=1)
dfRank = dfRank.drop(['FG3A_RANK', 'PLUS_MINUS_RANK', 'FGM_RANK', 'FTA_RANK'], axis=1)

###Regressão

Como a variável que queremos prever é categórica (FINAL = 0 iu 1) iremos utilizar a regressão logística:

In [183]:
y = dfEstat['FINAL']
xEstat = dfEstat.iloc[:,:-1]
xRank = dfRank.iloc[:,:-1]

Análise com as estatísticas:

In [184]:
logReg = LogisticRegression(max_iter=1000)
logReg.fit(xEstat, y)

predictsEstat = logReg.predict(xEstat)
confusion_matrix(y,predictsEstat)


array([[363,   1],
       [ 21,   5]])

Análise com os ranks:

In [185]:
logRegRank = LogisticRegression(max_iter=1000)
logRegRank.fit(xRank,y)

predictsRank = logRegRank.predict(xRank)
confusion_matrix(y,predictsRank)

array([[362,   2],
       [ 17,   9]])

Análise com todos os ranks:

In [186]:
logRegRank = LogisticRegression(max_iter=1000)
logRegRank.fit(dfRankComp.iloc[:,:-1],y)

predictsRank = logRegRank.predict(dfRankComp.iloc[:,:-1])
confusion_matrix(y,predictsRank)

array([[362,   2],
       [ 14,  12]])

Análise com os ranks mais importantes de acordo com o outro colab:

In [187]:
df = dfRank
x = df[['W_PCT_RANK','BLKA_RANK','FG_PCT_RANK','DREB_RANK','PTS_RANK','AST_RANK','FG3_PCT_RANK','BLK_RANK']]
y = df['FINAL']

logReg3 = LogisticRegression(max_iter=1000)
logReg3.fit(x,y)

predicts3 = logReg3.predict(x)
confusion_matrix(y,predicts3)

array([[360,   4],
       [ 20,   6]])